In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pickle
import time
from tqdm.notebook import tqdm

import torch
torch.set_default_tensor_type(torch.DoubleTensor)

from spatial_scene_grammars.constraints import *
from spatial_scene_grammars.nodes import *
from spatial_scene_grammars.rules import *
from spatial_scene_grammars.scene_grammar import *
from spatial_scene_grammars.visualization import *
from spatial_scene_grammars_examples.table.grammar import *
from spatial_scene_grammars.parsing import *
from spatial_scene_grammars.sampling import *
from spatial_scene_grammars.parameter_estimation import *
from spatial_scene_grammars.dataset import *

import meshcat
import meshcat.geometry as meshcat_geom

/home/nepfaff/src/spatial_scene_grammars/.venv/lib/python3.10/site-packages/torch/__init__.py:747: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:431.)
  _C._set_default_tensor_type(t)


In [2]:
if 'vis' not in globals():
    vis = meshcat.Visualizer()
vis.delete()
base_url = "http://127.0.0.1"
meshcat_url = base_url + ":" + vis.url().split(":")[-1]
print("Meshcat url: ", meshcat_url)
'''
from IPython.display import HTML
HTML("""
    <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; resize: both">
    <iframe src="{url}" style="width: 100%; height: 100%; border: none"></iframe>
</div>
""".format(url=meshcat_url))
'''

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Meshcat url:  http://127.0.0.1:7000/static/


'\nfrom IPython.display import HTML\nHTML("""\n    <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; resize: both">\n    <iframe src="{url}" style="width: 100%; height: 100%; border: none"></iframe>\n</div>\n""".format(url=meshcat_url))\n'

In [3]:
# Set up grammar and constraint set.
grammar = SpatialSceneGrammar(
    root_node_type = Table,
    root_node_tf = drake_tf_to_torch_tf(RigidTransform(p=[0.0, 0., 0.8]))
)
constraints = [
    ObjectsOnTableConstraint(),
    ObjectSpacingConstraint(),
    TallStackConstraint(),
    NumStacksConstraint()
]

def sample_realistic_scene(grammar, constraints, seed=None, skip_physics_constraints=False):
    if seed is not None:
        torch.random.manual_seed(seed)
    structure_constraints, pose_constraints = split_constraints(constraints)
    if len(structure_constraints) > 0:
        tree, success = rejection_sample_under_constraints(grammar, structure_constraints, 1000, detach=True)
        if not success:
            logging.error("Couldn't rejection sample a feasible tree config.")
            return None, None
    else:
        tree = grammar.sample_tree(detach=True)

    samples = do_fixed_structure_hmc_with_constraint_penalties(
        grammar, tree, num_samples=25, subsample_step=1,
        with_nonpenetration=False, zmq_url=vis.window.zmq_url,
        constraints=pose_constraints,
        kernel_type="NUTS", max_tree_depth=6, target_accept_prob=0.8, adapt_step_size=True,
        #kernel_type="HMC", num_steps=1, step_size=1E-1, adapt_step_size=False, # Langevin-ish
        structure_vis_kwargs={"with_triad": False, "linewidth": 30, "node_sphere_size": 0.02,
                             "alpha": 0.5}
    )

    # Step through samples backwards in HMC process and pick out a tree that satisfies
    # the constraints.
    good_tree = None
    best_bad_tree = None
    best_violation = None
    for candidate_tree in samples[::-1]:
        total_violation = eval_total_constraint_set_violation(candidate_tree, constraints)
        if total_violation <= 0.:
            good_tree = candidate_tree
            break
        else:
            if best_bad_tree is None or total_violation <= best_violation:
                best_bad_tree = candidate_tree
                best_violation = total_violation.detach()
            
    if good_tree == None:
        logging.error("No tree in samples satisfied constraints.")
        print("Best total violation: %f" % best_violation)
        print("Violations of best bad tree:")
        for constraint in constraints:
            print("constraint ", constraint, ": ", constraint.eval(best_bad_tree))
        return None, None

    if skip_physics_constraints:
        return None, good_tree

    feasible_tree = project_tree_to_feasibility(deepcopy(good_tree), do_forward_sim=True, timestep=0.001, T=1.)
    return feasible_tree, good_tree

In [4]:
test_tree, _ = sample_realistic_scene(grammar, constraints, seed=42)
if test_tree is not None:
    draw_scene_tree_contents_meshcat(test_tree, zmq_url=vis.window.zmq_url,  prefix="test_tree/contents")
    draw_scene_tree_structure_meshcat(test_tree, zmq_url=vis.window.zmq_url,  prefix="test_tree/structure")
# Save that scene out for use in another notebook
with open("example_feasible_sampled_scene.pickle", "wb") as f:
    pickle.dump(test_tree, f)

Initial trace log prob:  tensor(-3586.9977)


Sample: 100%|██████████| 37/37 [00:31,  1.18it/s, step size=9.59e-04, acc. prob=0.941]


In [5]:
# Produce dataset by sampling a bunch of environments.
# Save grammar state dict
#state_dict_file = "target_dataset_grammar_state_dict.torch"
state_dict_file = "structure_constraint_dataset_grammar_state_dict.torch"
print("Saving state dict to ", state_dict_file)
torch.save(grammar.state_dict(), state_dict_file)
# Try to collect a target number of examples, and save them out
#dataset_save_file = "target_dataset_examples.pickle"
dataset_save_file = "structure_constraint_examples.pickle"
N = 20
k = 0
pbar = tqdm(total=N, desc="Samples")
while k < N:
    tree, _ = sample_realistic_scene(grammar, constraints)
    if tree is not None:
        with open(dataset_save_file, "a+b") as f:
            pickle.dump(tree, f)
        k += 1
        pbar.update(k)
pbar.close()

Saving state dict to  structure_constraint_dataset_grammar_state_dict.torch


/home/nepfaff/src/spatial_scene_grammars/spatial_scene_grammars/sampling.py:444: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif kernel_type is "HMC":


Samples:   0%|          | 0/20 [00:00<?, ?it/s]

Initial trace log prob:  tensor(-7934.2943)


Sample: 100%|██████████| 37/37 [00:28,  1.29it/s, step size=9.11e-04, acc. prob=0.940]


...

Sample: 100%|██████████| 37/37 [00:32,  1.15it/s, step size=9.90e-04, acc. prob=0.955]


Initial trace log prob:  tensor(-1913.7468)


Sample: 100%|██████████| 37/37 [00:32,  1.16it/s, step size=1.37e-03, acc. prob=0.869]


Logfile: 

         S N O P T  7.4-1.2  (Feb 2015)
1
 
 SNMEMA EXIT 100 -- finished successfully
 SNMEMA INFO 104 -- memory requirements estimated
1
 
 Parameters

 Files
 -----
 Solution file..........         0       Old basis file ........         0       Standard input.........         5
 Insert file............         0       New basis file ........         0       (Printer)..............        10
 Punch file.............         0       Backup basis file......         0       (Specs file)...........         0
 Load file..............         0       Dump file..............         0       Standard output........         6

 Frequencies
 -----------
 Print frequency........       100       Check frequency........        60       Save new basis map.....       100
 Summary frequency......       100       Factorization frequency        50       Expand frequency.......     10000

 QP subproblems
 --------------
 QPsolver Cholesky......
 Scale tolerance........     0.900       Minor 

Sample: 100%|██████████| 37/37 [00:33,  1.11it/s, step size=5.12e-04, acc. prob=0.962]


In [6]:
print(tree.score())

tensor([-inf])


# Build sample population from various stages of em fitting process

In [7]:
fit_grammar = SpatialSceneGrammar(
    root_node_type = Table,
    root_node_tf = drake_tf_to_torch_tf(RigidTransform(p=[0.0, 0., 0.8]))
)
constraints = [
    ObjectsOnTableConstraint(),
    ObjectSpacingConstraint()
]
#fit_grammar.load_state_dict(torch.load("fit_grammar.torch"))
with open("fit_em.pickle", "rb") as fp:
    em = pickle.load(fp)
fit_grammar.load_state_dict(em.grammar_iters[-1])

<All keys matched successfully>

In [9]:
test_tree, _ = sample_realistic_scene(fit_grammar, constraints, seed=43, skip_physics_constraints=False)
if test_tree is not None:
    draw_scene_tree_contents_meshcat(test_tree, zmq_url=vis.window.zmq_url,  prefix="test_tree/contents")
    draw_scene_tree_structure_meshcat(test_tree, zmq_url=vis.window.zmq_url,  prefix="test_tree/structure")

Initial trace log prob:  tensor(-21602.9392)


Sample: 100%|██████████| 37/37 [00:33,  1.09it/s, step size=5.55e-04, acc. prob=0.944]


In [ ]:
# This one saves out the pre-fit examples, and doesn't bother with physics,
# since these samples are all over the place. It still does HMC to try to resolve
# the basic constraints (for better comparison with post-fit)
fit_grammar.load_state_dict(em.grammar_iters[0])
state_dict_file = "pre_fit_grammar_state_dict.torch"
print("Saving state dict to ", state_dict_file)
torch.save(fit_grammar.state_dict(), state_dict_file)
# Try to collect a target number of examples, and save them out
dataset_save_file = "pre_fit_grammar_draws.pickle"
N = 50
k = 0
pbar = tqdm(total=N, desc="Samples")
while k < N:
    print(k, N)
    _, nonfeasible_tree = sample_realistic_scene(fit_grammar, constraints, skip_physics_constraints=True)
    if nonfeasible_tree is not None:
        with open(dataset_save_file, "a+b") as f:
            pickle.dump((None, nonfeasible_tree), f)
        k += 1
        pbar.update(k)
pbar.close()

In [ ]:
# This one saves out the post-fit examples, before and after the physics projection step.
fit_grammar.load_state_dict(em.grammar_iters[-1])
state_dict_file = "post_fit_grammar_state_dict.torch"
print("Saving state dict to ", state_dict_file)
torch.save(fit_grammar.state_dict(), state_dict_file)
# Try to collect a target number of examples, and save them out
dataset_save_file = "post_fit_grammar_draws.pickle"
N = 50
k = 0
pbar = tqdm(total=N, desc="Samples")
while k < N:
    feasible_tree, nonfeasible_tree = sample_realistic_scene(fit_grammar, constraints, skip_physics_constraints=False)
    if feasible_tree is not None:
        with open(dataset_save_file, "a+b") as f:
            pickle.dump((feasible_tree, nonfeasible_tree), f)
        k = k + 1
        pbar.update(k)
pbar.close()

# Same thing, but baseline grammar

In [ ]:
from spatial_scene_grammars_examples.table.grammar_baseline import *

fit_grammar = SpatialSceneGrammar(
    root_node_type = TableBaseline,
    root_node_tf = drake_tf_to_torch_tf(RigidTransform(p=[0.0, 0., 0.8]))
)
constraints = [
    ObjectsOnTableConstraintBaseline(),
    ObjectSpacingConstraintBaseline()
]
#fit_grammar.load_state_dict(torch.load("fit_grammar.torch"))
with open("fit_em_baseline.pickle", "rb") as fp:
    em = pickle.load(fp)
fit_grammar.load_state_dict(em.grammar_iters[-1])

<All keys matched successfully>

In [ ]:
test_tree, _ = sample_realistic_scene(fit_grammar, constraints, seed=43, skip_physics_constraints=False)
if test_tree is not None:
    draw_scene_tree_contents_meshcat(test_tree, zmq_url=vis.window.zmq_url,  prefix="test_tree/contents")
    draw_scene_tree_structure_meshcat(test_tree, zmq_url=vis.window.zmq_url,  prefix="test_tree/structure")

Initial trace log prob:  tensor(-18354.6738)
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.


Sample: 100%|██████████| 37/37 [01:06,  1.81s/it, step size=6.44e-04, acc. prob=0.923]


In [ ]:
# This one saves out the pre-fit examples, and doesn't bother with physics,
# since these samples are all over the place. It still does HMC to try to resolve
# the basic constraints (for better comparison with post-fit)
fit_grammar.load_state_dict(em.grammar_iters[0])
state_dict_file = "baseline_pre_fit_grammar_state_dict.torch"
print("Saving state dict to ", state_dict_file)
torch.save(fit_grammar.state_dict(), state_dict_file)
# Try to collect a target number of examples, and save them out
dataset_save_file = "baseline_pre_fit_grammar_draws.pickle"
N = 30
k = 0
pbar = tqdm(total=N, desc="Samples")
while k < N:
    print(k, N)
    _, nonfeasible_tree = sample_realistic_scene(fit_grammar, constraints, skip_physics_constraints=True)
    if nonfeasible_tree is not None:
        with open(dataset_save_file, "a+b") as f:
            pickle.dump((None, nonfeasible_tree), f)
        k += 1
        pbar.update(k)
pbar.close()

Saving state dict to  baseline_pre_fit_grammar_state_dict.torch


Samples:   0%|          | 0/30 [00:00<?, ?it/s]

0 30
Initial trace log prob:  tensor(-6560784.8161)



Warmup:   0%|          | 0/37 [00:00, ?it/s]

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.


...

Sample: 100%|██████████| 37/37 [00:36,  1.00it/s, step size=1.58e-03, acc. prob=0.890]


29 30
Initial trace log prob:  tensor(-7064587.2389)



Warmup:   0%|          | 0/37 [00:00, ?it/s]

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.


Sample: 100%|██████████| 37/37 [00:21,  1.71it/s, step size=1.15e-03, acc. prob=0.957]


In [10]:
# This one saves out the post-fit examples, before and after the physics projection step.
fit_grammar.load_state_dict(em.grammar_iters[-1])
state_dict_file = "baseline_post_fit_grammar_state_dict.torch"
print("Saving state dict to ", state_dict_file)
torch.save(fit_grammar.state_dict(), state_dict_file)
# Try to collect a target number of examples, and save them out
dataset_save_file = "baseline_post_fit_grammar_draws.pickle"
N = 30
k = 0
pbar = tqdm(total=N, desc="Samples")
while k < N:
    feasible_tree, nonfeasible_tree = sample_realistic_scene(fit_grammar, constraints, skip_physics_constraints=False)
    if feasible_tree is not None:
        with open(dataset_save_file, "a+b") as f:
            pickle.dump((feasible_tree, nonfeasible_tree), f)
        k = k + 1
        pbar.update(k)
        print("At iter %d" % k)
        draw_scene_tree_contents_meshcat(feasible_tree, zmq_url=vis.window.zmq_url,  prefix="test_tree/contents")
        draw_scene_tree_structure_meshcat(feasible_tree, zmq_url=vis.window.zmq_url,  prefix="test_tree/structure")
pbar.close()

Saving state dict to  baseline_post_fit_grammar_state_dict.torch


Samples:   0%|          | 0/30 [00:00<?, ?it/s]

Initial trace log prob:  tensor(-14581.0879)


Sample: 100%|██████████| 37/37 [00:21,  1.71it/s, step size=7.95e-04, acc. prob=0.852]


At iter 1
Initial trace log prob:  tensor(-920.9405)


...

Sample: 100%|██████████| 37/37 [00:22,  1.63it/s, step size=9.72e-04, acc. prob=0.841]


At iter 24
Initial trace log prob:  tensor(229.0027)


Sample: 100%|██████████| 37/37 [00:21,  1.72it/s, step size=1.01e-03, acc. prob=0.772]


At iter 25
Initial trace log prob:  tensor(-2455.5355)


Sample: 100%|██████████| 37/37 [00:25,  1.43it/s, step size=9.62e-04, acc. prob=0.793]
